# TEsorter - https://github.com/zhangrengang/TEsorter
## Uma ferramenta para classificação de elementos transponíveis 
### Ele é codificado para LTR_retriever para classificar retrotransposons de repetição terminal longa (LTR-RTs) primeiro. Ele também pode ser usado para classificar quaisquer outros elementos transponíveis (TEs), incluindo elementos de Classe I e Classe II que são cobertos pelo banco de dados REXdb.
### Desde a versão v1.4, um modo GENOME é suportado para identificar domínios de proteína TE em todo o genoma.

In [ ]:
# Apos ter baixado o TEsorter

## 1 - Se entra na pasta TEsorter/TEsorter/database e baixa o arquivo Pfam-A.hmm.gz
wget -c https://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz 
gunzip Pfam-A.hmm.gz # descompacta o arquivo
sudo hmmpress # cria o banco de dados do HMM para o TEsorter rodar corretamente 

## 2 Entrar no link https://isfinder.biotoul.fr/scripts/search-db.php e baixar todos os fastas de origem Streptococcus agalactiae, dentro da mesma pasta especificada no 1 passo (TEsorter/TEsorter/database)
### Para baixar os fastas de origem Streptococcus agalactiae, é necessário buscar (search) Streptococcus agalactiae e depois acessar o numero de acesso (Accession Number)

# Apos ter feito os passos acimas, vamos criar os arquivos fasta de entrada para o TEsorter
## Criar e Ativar o conda bedtools
conda create -n bedtools
conda activate bedtools
conda install -c bioconda bedtools
### Apos isso e so necessario ativar o conda bedtools para rodar o bedtools

## Entrar na pasta do RepeatMasker
cd paht/RepeatMasker/RepeatMaske_folder
## EX:
cd /mnt/dados/victor_baca/outputs/RepeatMasker/SGEHI2015-95

### Utilizando Bedtools para gff virar fasta #############################################################
#### Primeiro é necessário transformar o arquivo gff em bed
awk -F'\t' '!/^#/ {print $1 "\t" $4-1 "\t" $5 "\t" $9}' arquivo.gff > arquivo.bed
awk -F'\t' '!/^#/ {print $1 "\t" $4-1 "\t" $5 "\t" $9}' GCA_020980155.1_ASM2098015v1_genomic.fna.out.gff > GBS95.bed

##### Encontrar o genoma de referência
find path/genoma -iname ID_NCBI*
##### EX:
find /mnt/dados/victor_baca/incomplete_genomes -iname GCA_020980155.1_ASM2098015v1*

##### Utilizando o bedtools para transformar o arquivo bed em fasta #########################################
bedtools getfasta -fi genoma.fna -bed arquivo.bed -fo saida.fasta
### EX:
bedtools getfasta -fi /mnt/dados/victor_baca/incomplete_genomes/GCA_020980155.1_ASM2098015v1_genomic.fna -bed GBS95.bed -fo GBS95.fasta

# TEsorter
## 1 - Entrar na pasta do TEsorter 
cd /mnt/dados/victor_baca/tools/TEsorter/TEsorter
### E criar um diretório para os resultados
mkdir outputsgffbedtools

## 2 - Encontrar o arquivo fasta gerado pelo bedtools
find path -iname *.fasta
## EX:
find /mnt/dados/victor_baca/outputs/RepeatMasker/SGEHI2015-95 -iname *.fasta

## 3 - Rodar o TEsorter
TEsorter --db-hmm /mnt/dados/victor_baca/tools/TEsorter/TEsorter/database/Pfam-A.hmm -p 5 -cov 80 -eval 0.0001 -rule 80-80-80 path/file.fasta

## --db-hmm: banco de dados de domínios de proteínas Pfam A (arquivo .hmm) 
## -p 5: número de threads
## -cov 80: cobertura mínima do domínio
## -eval 0.0001: valor de e-valor máximo
## -rule 80-80-80: regra de classificação de TE

### EX:
TEsorter --db-hmm /mnt/dados/victor_baca/tools/TEsorter/TEsorter/database/Pfam-A.hmm -cov 80 -eval 0.0001 -rule 80-80-80 /mnt/dados/victor_baca/outputs/RepeatMasker/SGEHI2015-95/SGEHI2015-95.fasta


# Visualizar os resultados
## dom.tsv - valor da probabilidade de ser TE -> anotar se todos os valores são maiores que 0.8 e anotar o dominio (motivo catalítico) Pfam detectado (ex: DDE - HUH) das transposases

FILE="SGEHI2015-95.fasta.Pfam-A.dom.tsv"; \
echo -e "\nDomínios Pfam detectados:"; \
awk -F'\t' 'NR>1 {split($1,a,"|"); print a[2]}' "$FILE" | sort | uniq -c; \
echo -e "\nVerificação de probability > 0.8:"; \
awk -F'\t' 'NR>1 {if ($5 <= 0.8) {print "ALERTA: " $1 " → " $5; count++}} END {if (count == 0) print "Todos têm probability > 0.8 ✅"}' "$FILE"